## Notes on Dexter's SAS model statements

### First a few data notes:

### At this point it is a little tricky to follow what Dexter has done, that we have also done, or, do not want to do.

### First, Dexter omitted three populations, RWI, CDR and VES for various reasons, but I repaired all these except VES, which is a population where I still need to trace down the X/Y coords. Still, I have not deleted the VES plants.

### Second, Dexter and I made much the same decisions about missing values and obvious corrections to the data.

### Third, for size related traits, Dexter divided the sums across censuses (e.g. number of leaves) by the longevity of the plant. This gives a sort of average phenotype for an average census, but it strikes me that all those traits are now correlated wtih longevity in some way and therefore each other. So let's not do that.

### Finally, Dexter used PCA on neutral markers and environmental variables, then used ANOVA to get the residuals of the traits. He then analyzed were the residuals from this ANOVA for continent effects etc., thus trying to remove effects the environment, gender, neutral markers etc. I have some issues with this because some of the differences between continents for example may be environmental and we would be removing effects on phenotype that we would like to study. Anyway, here is the flow of all that.

### Note below that we had previously run a PCA on the AFLP data. The variables to be used later are the first two PCAs of the genetic data

In [ ]:
#/*      Import the AFLP data set         */
#DATA LAT_PCO; DEL=BYTE(09);
#INFILE 'C:\Documents and Settings\drt_lab\My Documents\My SAS Files\9.1\common_garden\Common Gardens 2009\LAT_PCO_020709.txt' DELIMITER=DEL;
#INPUT REGION $ POP $ LAT LONG IND $ GENPC1 GENPC2 PCO3 PCO4 PCO5 PCO6;
#DROP REGION LAT LONG IND PCO3 PCO4 PCO5 PCO6;
#RUN;

### Next, we did a PCA on the following environmental variables ...

In [ ]:
#*        Input the environmental data        */
#DATA ENV_LAT; DEL=BYTE(09);
#INFILE 'C:\Documents and Settings\drt_lab\My Documents\My SAS Files\9.1\common_garden\Common Gardens 2009\LAT_GIS_DATA.TXT' DELIMITER=DEL;
#INPUT POP $ LAT LONG AVGANNTMP AVGTEMPCLDQTR AVGANNPCP PCPDRYMNTH PCPSEASON PCPWRMQTR TMPSEASON MAXTMPWRMMNTH MINTMPCLDMNTH;
#RUN;
#/*** -Environmental Acronyms described-
#AVGANNTMP=average annual temp
#AVGTEMPCLDQTR=average temperature in coldest quarter
#AVGANNPCP=average annual percipitation
#PCPDRYMNTH=average percipitation in driest month
#PCPSEASON=percipitation seasonality (CV)
#PCPWRMQTR=average percipitation in warmest quarter;
#TMPSEASON=temperature seasonality (SD across months ◊ 100)
#MAXTMPWRMMNTH=maximum temperature in warmest month;
#MINTMPCLDMNTH=minimum temperature in coldest month      ***/

### Here is the model statement for the PCA ... if we want to do something like this, I can use my scripts

In [ ]:
#/*** The following code creates orthogonal principal componenets of the environmental data, of which only the first two PCs will be retained ***/
#DM 'CLEAR LOG';
#PROC PRINCOMP DATA=ENV_LAT NOPRINT OUT=ENVPCA PREFIX=ENV_PC;    
#VAR AVGANNTMP AVGANNPCP PCPDRYMNTH PCPSEASON PCPWRMQTR TMPSEASON MAXTMPWRMMNTH MINTMPCLDMNTH;
#RUN; /*  creates multiple principal components of the 8 environmental variables, ortogonal data set named ENVPCA  */

#DATA ENVIRONMENT;  /*  Data step retains only the first two PCs of environmental data. Abbreviated data set renamed ENVIRONMENT */
#SET ENVPCA;
#KEEP POP LAT LONG ENV_PC1 ENV_PC2;  RUN;

### The resulting data have the phenotypic variables, the class variables, lats and longs, the first two PCs describing the environment, and the first two PCs describing the nuclear genetic markers.

### The phenotypic variables were log transformed, x=log(x+1)

### Next, a mixed model ANOVA was run on each trait, generating residuals for the actual analysis. 

In [ ]:
#/* TRAIT 1, STEM LEAVES*/  /*DM 'CLEAR OUTPUT';*/
#PROC MIXED DATA=ALLMERGED COVTEST; TITLE 'Stem leaves-weighted by lifespan-LN+1'; WHERE sumALIVE>0; 
#CLASS SEX;
#MODEL sumSTEM_LVs=SEX LAT LONG ENV_PC1 ENV_PC2/OUTP=TEMPRESID1 RESIDUAL; RUN;
#DATA TRAIT1; SET TEMPRESID1; KEEP POP ORIGIN DEST SITE TRT PLOT PLANTID SEX rsumSTEM_LVs; rsumSTEM_LVs=RESID; RUN;
#PROC SORT; BY PLANTID; RUN;

### The model statement:
### MODEL sumSTEM_LVs=SEX LAT LONG ENV_PC1 ENV_PC2/OUTP=TEMPRESID1
### means that the variable TEMPRESID1 is the residual of stem leaves, with the effect of Sex, lat, long, ENVPc1 and ENVPC2 removed. I do not see the genetic PCs in this model statement.
### This basic structure was used for 10 variables, and the resulting merged data had the class variables and the 10 variables which were actually the residuals from above.

### ____________________________________________
### Yet another round of PCAs were done, this time on the variables themselves.
### FOUR different PCAs were done on a priori defined traits, size related traits, defense related traits, life-history related traits, and all traits. Note that these PCAs were done on the residuals from above, and the ANOVAs of interest were done on the first two PCAs.

In [ ]:
# PCA for size related traits

#/* The following PROC PRINCOMPs produce principal components of the various traits */
#DM 'CLEAR LOG'; DM 'CLEAR OUTPUT'; /*Principal Components on Size Traits*/
#PROC PRINCOMP DATA=MERGEDRESIDS  OUT=PCSIZE PREFIX=PCSIZE_  ;
#TITLE 'PC for Size traits: Stem lvs, Basal lvs, Bolt no, Bolt ht';
#VAR rsumSTEM_LVs rsumBASAL_LVs rsumNUM_BOLTS rsumBOLT_HT;
#RUN;

In [ ]:
# PCA for defense related traits

#DM 'CLEAR LOG'; DM 'CLEAR OUTPUT'; /*Principal Components on Defense Traits*/
#PROC PRINCOMP DATA=MERGEDRESIDS  OUT=PCDEF  PREFIX=PCDEF_ ;
#TITLE 'PC for Defense traits: Leaf herbivory, Gen enemy load, Spec enemy load';
#VAR rsumLH rsumGEN_ENMY rsumSPEC_ENMY;
#RUN;

In [ ]:
# PCA for life-history traits

#DM 'CLEAR LOG'; DM 'CLEAR OUTPUT'; /*Principal Components on Life History Traits*/
#PROC PRINCOMP DATA=MERGEDRESIDS  OUT=PCLIFE PREFIX=PCLIFE_ ;
#TITLE 'PC for Life History traits: Longevity, Time to Reproduction';
#VAR rLifespn1 rDelyRep1;
#RUN;

In [ ]:
# PCA for all traits

#/*Principal Components on ALL (Size, Def, Life H) Traits
#DM 'CLEAR LOG'; DM 'CLEAR OUTPUT'; 
#PROC PRINCOMP DATA=MERGEDRESIDS  OUT=PCALL PREFIX=PCALL_ ;
#TITLE 'All: Stem & Basal lvs, Bolt no & ht; LH, genENMY, specENMY; Longevity, Time to Reproduction'; 
#VAR rsumSTEM_LVs rsumBASAL_LVs rsumNUM_BOLTS rsumBOLT_HT rsumLH rsumGEN_ENMY rsumSPEC_ENMY rsumALIVE rsumREPROD_SPD rTOTFLR; RUN;

### So in the end, we log-transformed the variables, calculated the residuals of some nuisance variables including their genders and estimates of their abiotic environment, then took the PCs of those residuals and analyzed them. It makes me tired just thinking about it and I think we should start with the basics ...

### Below is an ANOVA involving an analysis of one of these PCAS.

In [4]:
#/*** The following PROC MIXEDs analyze the -SIZE- Principal Components ***/
#PROC SORT DATA=PCSIZE; BY ORIGIN DEST TRT POP SITE PLOT; RUN;
#DM 'CLEAR OUTPUT'; DM 'CLEAR LOG';
#PROC MIXED DATA=PCSIZE COVTEST;
#TITLE1 'PC SIZE 1'; TITLE2 'Plants with longevity=0 excluded (transplant shock)'; WHERE POP^="CDR";
#CLASS Origin Dest Trt Pop Site Plot;
#MODEL PCSIZE_1=Origin Dest Trt Origin*Dest Origin*Trt Dest*Trt Origin*Dest*Trt;
#RANDOM Pop(Origin) Site(Dest) Plot(Site) Trt*Plot;
#LSMEANS Origin; LSMEANS Dest; LSMEANS Trt; LSMEANS Origin*Dest; LSMEANS Origin*Trt; LSMEANS Dest*Trt; LSMEANS Origin*Dest*Trt; RUN; QUIT;

### The CLASS variables are, in our language cont_orig, cont_dest, treatment, pop, site, and plot. The random effects are pop(origin) [pop nested within origin], site(dest) [site nested within destination], Plot(site) [plot nested within site] and treatment*plot [i.e. all combinations of treatments and plots comprise a random effect]

### MODEL PCSIZE_1=Origin Dest Trt OriginxDest OriginxTrt DestxTrt OriginxDestxTrt;
### Notice how the model includes all the CLASS variables and none of the RANDOM variables.
### Our model would have to include SEX as a CLASS variable unless we do the residuals thing. But the thing kind of explodes. Does this mean our model statement is Y = origin destination treatmentment sex (all_two way_interactions) (all_three_ways) and (the one four way interaction)?
### It would help for us to figure out which ANOVA effects we are actually interested in ....
